In [3]:
import cv2
from ultralytics import YOLO
import firebase_admin
from firebase_admin import credentials, db
import time
from threading import Timer

# Global flag to indicate whether the program is running
program_running = True

# Check if Firebase app is already initialized
if not firebase_admin._apps:
    # Initialize Firebase
    cred = credentials.Certificate("feeder-d63d2-firebase-adminsdk-ybhos-10e79d725f.json")
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://feeder-d63d2-default-rtdb.firebaseio.com/'
    })

# Load the YOLOv8 model
model = YOLO("best_model.pt")

# Open the webcam (use 0 for the default webcam)
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Define the colors for each label
label_colors = {
    "Unseated": (0, 255, 0),  # Green
    "Seated": (0, 0, 255),    # Red
}

def draw_boxes(frame, boxes, labels):
    for box, label in zip(boxes, labels):
        color = label_colors[label]
        x1, y1, x2, y2 = map(int, box)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

def display_label_count(frame, label_counts):
    y_offset = 30
    for label, count in label_counts.items():
        text = f"{label}: {count}"
        cv2.putText(frame, text, (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        y_offset += 30

# Function to send data to Firebase
def send_data_to_firebase(label_counts):
    global program_running
    if program_running:
        ref = db.reference('label_counts')
        timestamp = int(time.time())
        ref.child(str(timestamp)).set(label_counts)

# Initialize label_counts
label_counts = {"Unseated": 0, "Seated": 0}

# Timer function to send data every 30 seconds
def start_timer():
    global program_running
    if program_running:
        Timer(5, start_timer).start()
        send_data_to_firebase(label_counts)

# Start the timer
start_timer()

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the webcam
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True)

        # Extract the bounding boxes and labels
        boxes = results[0].boxes.xyxy.numpy()
        if hasattr(results[0], 'names'):
            label_ids = results[0].boxes.cls.numpy()
            labels = [results[0].names[int(label_id)] for label_id in label_ids]
        else:
            labels = []

        # Draw the bounding boxes with label-specific colors
        draw_boxes(frame, boxes, labels)

        # Reset label_counts
        label_counts = {"Unseated": 0, "Seated": 0}

        # Count the number of occurrences of each label
        for label in labels:
            if label in label_counts:
                label_counts[label] += 1

        # Display the label counts on the frame
        display_label_count(frame, label_counts)

        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if there's an error reading the frame
        print("Error: Could not read frame from webcam.")
        break

# Set the program_running flag to False to stop sending data to Firebase
program_running = False

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()

2024-07-07 20:52:36.038 Python[29295:559464] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.



0: 480x800 (no detections), 226.4ms
Speed: 4.4ms preprocess, 226.4ms inference, 1126.9ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 (no detections), 218.5ms
Speed: 3.3ms preprocess, 218.5ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 (no detections), 208.6ms
Speed: 3.1ms preprocess, 208.6ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 (no detections), 207.8ms
Speed: 2.9ms preprocess, 207.8ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 (no detections), 212.7ms
Speed: 3.7ms preprocess, 212.7ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 (no detections), 218.6ms
Speed: 3.6ms preprocess, 218.6ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 (no detections), 206.5ms
Speed: 3.2ms preprocess, 206.5ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 800)

0: 480x800 (no detections), 208.7ms
Speed: 3.1ms pr